In [1]:
import pandas as pd
import polars as pl
import numpy as np

from m3_model_params import lgb_params_1
from m4_kaggle_feats import *
from m5_sb_models import *
import warnings
warnings.filterwarnings("ignore")

data_path     = 'kaggle/input/linking-writing-processes-to-writing-quality/'
train_logs    = pl.scan_csv(f'{data_path}/train_logs.csv')
test_logs    = pl.scan_csv(f'{data_path}/test_logs.csv')
train_scores = pl.scan_csv(f'{data_path}/train_scores.csv')

In [2]:
tr, ts = cursor_position_change_gap(train_logs, test_logs)
ts.collect()

Engineering cursor position data


id,cursor_position_change100_max,cursor_position_change100_min,cursor_position_change100_mean,cursor_position_change100_std,cursor_position_change100_quantile,cursor_position_change100_median,cursor_position_change100_sum,cursor_position_change100_skew,cursor_position_change100_kurt
str,i64,i64,f64,f64,f64,f64,i64,f64,f64
"""0000aaaa""",null,null,null,null,null,null,0,null,null
"""4444cccc""",null,null,null,null,null,null,0,null,null
"""2222bbbb""",null,null,null,null,null,null,0,null,null


In [32]:
tr,ts = categorical_stats(train_logs, test_logs)
tr.collect()

id,event_id_max,up_time_max,activity_n_unique,down_event_n_unique,up_event_n_unique,text_change_n_unique
str,i64,i64,u32,u32,u32,u32
"""001519c8""",2557,1801969,7,12,12,17
"""0022f953""",2454,1788969,5,17,17,12
"""0042269b""",4136,1771669,4,13,18,19
"""0059420b""",1556,1404469,5,15,15,10
"""0075873a""",2531,1662472,3,11,11,9
"""0081af50""",2211,1778916,4,11,11,11
"""0093f095""",1765,1768197,3,13,13,9
"""009e23ab""",2353,1799303,4,17,17,12
"""00e048f1""",1585,1797498,3,8,8,6


In [ ]:
print("Engineering statistical summaries for features")
feats_stat = [
    ('event_id', ['max']),
    ('up_time', ['max']),
    ('action_time', ['max', 'min', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt]),
    ('activity', ['nunique']),
    ('down_event', ['nunique']),
    ('up_event', ['nunique']),
    ('text_change', ['nunique']),
    ('cursor_position', ['nunique', 'max', 'quantile', 'sem', 'mean']),
    ('word_count', ['nunique', 'max', 'quantile', 'sem', 'mean'])]

In [ ]:

def make_feats(self, df):
        gaps = [1, 2, 3, 5, 10, 20, 50, 100]
        feats = pd.DataFrame({'id': df['id'].unique().tolist()})
        
        print("Engineering time data")
        for gap in self.gaps:
            df[f'up_time_shift{gap}'] = df.groupby('id')['up_time'].shift(gap)
            df[f'action_time_gap{gap}'] = df['down_time'] - df[f'up_time_shift{gap}']
        df.drop(columns=[f'up_time_shift{gap}' for gap in self.gaps], inplace=True)

        print("Engineering cursor position data")
        for gap in self.gaps:
            df[f'cursor_position_shift{gap}'] = df.groupby('id')['cursor_position'].shift(gap)
            df[f'cursor_position_change{gap}'] = df['cursor_position'] - df[f'cursor_position_shift{gap}']
            df[f'cursor_position_abs_change{gap}'] = np.abs(df[f'cursor_position_change{gap}'])
        df.drop(columns=[f'cursor_position_shift{gap}' for gap in self.gaps], inplace=True)

        print("Engineering word count data")
        for gap in self.gaps:
            df[f'word_count_shift{gap}'] = df.groupby('id')['word_count'].shift(gap)
            df[f'word_count_change{gap}'] = df['word_count'] - df[f'word_count_shift{gap}']
            df[f'word_count_abs_change{gap}'] = np.abs(df[f'word_count_change{gap}'])
        df.drop(columns=[f'word_count_shift{gap}' for gap in self.gaps], inplace=True)
        
        print("Engineering statistical summaries for features")
        feats_stat = [
            ('event_id', ['max']),
            ('up_time', ['max']),
            ('action_time', ['max', 'min', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt]),
            ('activity', ['nunique']),
            ('down_event', ['nunique']),
            ('up_event', ['nunique']),
            ('text_change', ['nunique']),
            ('cursor_position', ['nunique', 'max', 'quantile', 'sem', 'mean']),
            ('word_count', ['nunique', 'max', 'quantile', 'sem', 'mean'])]
        
        for gap in self.gaps:
            feats_stat.extend([
                (f'action_time_gap{gap}', ['max', 'min', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt]),
                (f'cursor_position_change{gap}', ['max', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt]),
                (f'word_count_change{gap}', ['max', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt])
            ])
        
        pbar = tqdm(feats_stat)
        for item in pbar:
            colname, methods = item[0], item[1]
            for method in methods:
                pbar.set_postfix()
                if isinstance(method, str):
                    method_name = method
                else:
                    method_name = method.__name__
                pbar.set_postfix(column=colname, method=method_name)
                tmp_df = df.groupby(['id']).agg({colname: method}).reset_index().rename(columns={colname: f'{colname}_{method_name}'})
                feats = feats.merge(tmp_df, on='id', how='left')

        print("Engineering activity counts data")
        tmp_df = self.activity_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering event counts data")
        tmp_df = self.event_counts(df, 'down_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        tmp_df = self.event_counts(df, 'up_event')
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering text change counts data")
        tmp_df = self.text_change_counts(df)
        feats = pd.concat([feats, tmp_df], axis=1)
        
        print("Engineering punctuation counts data")
        tmp_df = self.match_punctuations(df)
        feats = pd.concat([feats, tmp_df], axis=1)

        print("Engineering input words data")
        tmp_df = self.get_input_words(df)
        feats = pd.merge(feats, tmp_df, on='id', how='left')

        print("Engineering ratios data")
        feats['word_time_ratio'] = feats['word_count_max'] / feats['up_time_max']
        feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max']
        feats['event_time_ratio'] = feats['event_id_max']  / feats['up_time_max']
        feats['idle_time_ratio'] = feats['action_time_gap1_sum'] / feats['up_time_max']

        return feats